In [1]:
# 허깅페이스 추론
# 파이프라인을 이용하면 언어, 컴퓨터비전, 오디오, 멀티모달 태스크에 대한 추론을 위해 허깅페이스의 모든 모델을 쓸 수 있다.

In [ ]:
# 파이프라인 사용방법
# 파이프라인은 태스크에 맞게 추론이 가능한 기본모델과 전처리 클래스를 자동으로 로딩한다. 
from transformers import pipeline
generator = pipeline(task="automatic-speech-recognition")
generator("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")
# 기능을 입력하면 각각에 맞게 로딩이 된다. 태스크 목록은 튜토리얼 1장에 위치한다. 

In [ ]:
# 특정모델 지정 사용
generator = pipeline(model="openai/whisper-large")
generator("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")
# whisper모델을 이용한 음성-문자변환을 사용하였다. 

In [ ]:
# 다중입력
generator(
    [
        "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac",
        "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/1.flac",
    ]
)
# 입력이 여러 개 있는 경우 리스트형태로 투입이 가능하다. 

In [ ]:
# 매개변수 설정
# 기본. 
generator(model="openai/whisper-large", my_parameter=1)
# out = generate(...)  # 파라미터는 1일 것이다..
# out = generate(..., my_parameter=2)  # 파라미터는 2일 것이다. 
# out = generate(...)  # 파라미터는 다시 1로 돌아올 것이다. 

In [ ]:
# 기기(device)
# 파이프라인을 작동시키는 기기를 설정할 수 있다. 
generator(model="openai/whisper-large", device=0) 
# device = 0인 경우 GPU를 사용하며 device = -1인 경우 CPU를 사용한다. 
# 만약 모델이 GPU 하나에 돌아가기 어려운 경우 "auto"모드를 사용할 수 있다. 
# 여기서 엑셀러레이트가 자동으로 설정해주므로 엑셀러레이트를 설치해야 한다. 
#!pip install accelerate
generator(model="openai/whisper-large", device_map="auto")

In [ ]:
# 배치사이즈
# 한번에 훈련시키는 사이즈를 지정한다. 
generator(model="openai/whisper-large", device=0, batch_size=2)
audio_filenames = [f"audio_{i}.flac" for i in range(10)]
texts = generator(audio_filenames)
# 파이프라인은 일괄처리의 복잡한 부분을 줄여준다. 

In [ ]:
# 특정 태스크용 매개변수
generator = pipeline(model="facebook/wav2vec2-large-960h-lv60-self", return_timestamps="word")
generator("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")
# 각 태스크마다 구현할 때 유연성과 옵션을 제공하기 위해 태스크용 매개변수가 있다. 
# 예를 들어 위와같이 태스크를 구성하면 단어마다 자막처럼 시간변수가 할당된다. 단어를 말한 시점도 출력한다. 
# {'text': 'I HAVE A DREAM BUT ONE DAY THIS NATION WILL RISE UP AND LIVE OUT THE TRUE MEANING OF ITS CREED', 
# 'chunks': [{'text': 'I', 'timestamp': (1.22, 1.24)}, {'text': 'HAVE', 'timestamp': (1.42, 1.58)}, 
# {'text': 'A', 'timestamp': (1.66, 1.68)}, {'text': 'DREAM', 'timestamp': (1.76, 2.14)}, 
# {'text': 'BUT', 'timestamp': (3.68, 3.8)}, {'text': 'ONE', 'timestamp': (3.94, 4.06)}, 
# {'text': 'DAY', 'timestamp': (4.16, 4.3)}, {'text': 'THIS', 'timestamp': (6.36, 6.54)}, 
# {'text': 'NATION', 'timestamp': (6.68, 7.1)}, {'text': 'WILL', 'timestamp': (7.32, 7.56)}, 
# {'text': 'RISE', 'timestamp': (7.8, 8.26)}, {'text': 'UP', 'timestamp': (8.38, 8.48)}, 
# {'text': 'AND', 'timestamp': (10.08, 10.18)}, {'text': 'LIVE', 'timestamp': (10.26, 10.48)}, 
# {'text': 'OUT', 'timestamp': (10.58, 10.7)}, {'text': 'THE', 'timestamp': (10.82, 10.9)}, 
# {'text': 'TRUE', 'timestamp': (10.98, 11.18)}, {'text': 'MEANING', 'timestamp': (11.26, 11.58)}, 
# {'text': 'OF', 'timestamp': (11.66, 11.7)}, {'text': 'ITS', 'timestamp': (11.76, 11.88)}, 
# {'text': 'CREED', 'timestamp': (12.0, 12.38)}]}
#  return_timestamps="word"를 사용하여 단어마다 말한 시점이 출력되었다. 

In [ ]:
# 메모리 적재 줄이고 빠르게 작업하는 비결
# 데이터세트 이터레이트는 다음과 같이 진행할 수 있다. 
def data():
    for i in range(1000):
        yield f"My example {i}"


pipe = pipe(model="gpt2", device=0)
generated_characters = 0
for out in pipe(data()):
    generated_characters += len(out["generated_text"])
# 이터레이터는 결과를 호출마다 생성하고 파이프라인은 입력이 순회할 수 있는 자료구조임을 자동으로 인식하여 GPU에서 
# 기존데이터가 처리되는 동안 새로운 데이터를 가져온다. 이때 전체 데이터세트를 메모리에 적재하지 않고 gpu에서
# 빠르게 작업을 할 수 있는 방법이다. 증가/감소하는 데이터는 배치사이즈 크기만큼 증가/감소가 일어난다. 

In [2]:
# 비전 파이프라인 
from transformers import pipeline

vision_classifier = pipeline(model="google/vit-base-patch16-224")
preds = vision_classifier(
    images="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
)
preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
preds
# 비전 태스크를 위해 파이프라인을 사용한 예시이다. 
# 모델 다운로드 -> 사진 다운로드 -> 훈련 -> 결과확인
# 스라소니 사진임을 명확하게 보여준다. 

'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg'

In [ ]:
# 텍스트 파이프라인
from transformers import pipeline

# 이 모델은 `zero-shot-classification` 모델이다.
# 텍스트를 분류하지만 상상할 수 있는 라벨을 자유롭게 선택할 수 있다.
classifier = pipeline(model="facebook/bart-large-mnli")
classifier(
    "I have a problem with my iphone that needs to be resolved asap!!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
)
# 라벨을 설정했고 라벨 중 가장 가능성이 높은 라벨이 선택된다. 

In [ ]:
# 멀티모달 파이프라인
# 즉 오디오, 비디오, 텍스트와 같은 데이터를 위한 파이프라인이다.
# 시각적 질의응답 태스크는 텍스트와 이미지를 모두 사용할 수 있다. 
# https://huggingface.co/spaces/impira/docquery/resolve/2359223c1837a7587402bda0f2643382a6eefeab/invoice.png
# 위의 명세서에서 거래명세서 번호를 묻는다면?
from transformers import pipeline

vqa = pipeline(model="impira/layoutlm-document-qa")
vqa(
    image="https://huggingface.co/spaces/impira/docquery/resolve/2359223c1837a7587402bda0f2643382a6eefeab/invoice.png",
    question="What is the invoice number?",
)
# 이와같은 코드를 통해 us-001임을 바로 알 수 있다. 

In [ ]:
from datasets import TaskTemplate